<a href="https://colab.research.google.com/github/ict3104-p2-team08/ict3104-p2-team08/blob/dev/3104_T08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity Detection ML Test

## 1. Prerequisites

This section provides the setting up of the environment

In [ ]:
!pip install torch -f https://download.pytorch.org/whl/cu113/torch-1.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchaudio -f https://download.pytorch.org/whl/cu113/torchaudio-0.12.1%2Bcu113-cp37-cp37m-linux_x86_64.whl
!pip install torchsummary==1.5.1 torchtext==0.13.1 torchvision -f https://download.pytorch.org/whl/cu113/torchvision-0.13.1%2Bcu113-cp37-cp37m-linux_x86_64.whl

In [ ]:
!git clone https://github.com/v-iashin/video_features.git
!pip install omegaconf==2.0.6

In [ ]:
# Imports widgets for jupyter
try:
    import piplite
    piplite.install(['ipywidgets'])
except ImportError:
    pass

## 2. Toggling of pipelines

This section shows the toggling of dependencies between TSU and STEP

In [ ]:
#insert code here

## 3. Data exploration

This section load and display video data from the Toyota Smarthome (TSU) project

In [ ]:
#import tsu video files
import os
FOLDER_PATH = 'input_files'
ROOT_PATH = '/content/data'

file_list = os.listdir(os.path.join(ROOT_PATH, FOLDER_PATH))

In [ ]:
#tsu video files to display in dropdown
from ipywidgets import  Dropdown, widgets

w = widgets.Dropdown(
    options= file_list,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(change['new'])

w.observe(on_change)

display(w)

In [ ]:
#display video 
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/data/input_files/%' + w.value

locate= "" 

for i in video_path: 
   if i=='%': 
       pass
   else: 
       locate+=i 

def show_video(locate, video_width = 500):
  video_file = open(locate, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
  
show_video(locate)

In [ ]:
#only run it for the first time
%cd video_features

In [ ]:
#extract to I3D feature file
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

# Select the feature type
feature_type = 'i3d'
args = OmegaConf.load(build_cfg_path(feature_type))
args.video_paths = ['/content/data/input_files/P02T02C06.mp4']
args.flow_type = 'raft'

extractor = ExtractI3D(args)
print(locate)

# Extract features
for video_path in args.video_paths:
    print(f'Extracting for {video_path}')
    feature_dict = extractor.extract(video_path)
    [(print(k), print(v.shape), print(v)) for k, v in feature_dict.items()]

## 4. Inference with pretrained model

This section shows

In [ ]:
#insert codes here

## 5. Testing and evaluating trained models

This section shows

In [ ]:
#insert codes here

## 6. Training of new model

This section shows

In [ ]:
#insert codes here